In [11]:
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [12]:
annotations_file = 'annotations_imSize.csv'

In [13]:
anno = pd.read_csv(annotations_file)
anno.head()

,filename,x1,y1,x2,y2,classID
0,01/image.000935.jp2,1346.82,246.76,1582.12,484.41,65
1,01/image.000936.jp2,1164.88,237.94,1324.29,397.35,65
2,01/image.000937.jp2,1077.29,232.49,1196.90,352.10,65
3,05/image.000938.jp2,1346.49,471.65,1416.46,538.50,34
4,05/image.000939.jp2,1466.78,500.63,1549.77,581.59,34


In [14]:
# Get filenames from annotations file and get size of each image

width_list = []
height_list = []

for i in anno["filename"]:
    image = Image.open(i)

    width, height = image.size
    
    width_list.append(width)
    height_list.append(height)

# Insert width and height of images to annotations data frame
anno['width'] = width_list
anno['height'] = height_list

anno.head()


,filename,x1,y1,x2,y2,classID,width,height
0,01/image.000935.jp2,1346.82,246.76,1582.12,484.41,65,1628,1236
1,01/image.000936.jp2,1164.88,237.94,1324.29,397.35,65,1628,1236
2,01/image.000937.jp2,1077.29,232.49,1196.90,352.10,65,1628,1236
3,05/image.000938.jp2,1346.49,471.65,1416.46,538.50,34,1628,1236
4,05/image.000939.jp2,1466.78,500.63,1549.77,581.59,34,1628,1236


In [15]:
anno["width"].value_counts()


1628    5463
Name: width, dtype: int64

In [16]:
anno["height"].value_counts()

1236    5463
Name: height, dtype: int64

In [17]:
def read_image(path):
    return cv2.cvtColor(cv2.imread(str(path)), cv2.COLOR_BGR2RGB)

In [18]:
def create_mask(bb, x):
    """Creates a mask for the bounding box of same shape as image"""
    rows,cols,*_ = x.shape
    Y = np.zeros((rows, cols))
    bb = bb.astype(np.int)
    Y[bb[0]:bb[2], bb[1]:bb[3]] = 1.
    return Y


def mask_to_bb(Y):
    """Convert mask Y to a bounding box, assumes 0 as background nonzero object"""
    cols, rows = np.nonzero(Y)
    if len(cols)==0: 
        return np.zeros(4, dtype=np.float32)
    top_row = np.min(rows)
    left_col = np.min(cols)
    bottom_row = np.max(rows)
    right_col = np.max(cols)
    return np.array([left_col, top_row, right_col, bottom_row], dtype=np.float32)

In [19]:
def create_bb_array(x):
    """Generates bounding box array from a anno row"""
    return np.array([x[2],x[1],x[4],x[3]])

In [23]:
1236/2

618.0

In [24]:
def resize_image_bb(read_path,write_path,bb,sz):
    """Resize an image and its bounding box and write image to new path"""
    im = read_image(read_path)
    im_resized = cv2.resize(im, (814, 618))
    Y_resized = cv2.resize(create_mask(bb, im), (814, 618))
    new_path = r'{}/{}'.format(write_path,read_path)
    print(new_path)
    cv2.imwrite(new_path, cv2.cvtColor(im_resized, cv2.COLOR_RGB2BGR))
    return new_path, mask_to_bb(Y_resized)

In [25]:
#Populating Training DF with new paths and bounding boxes
new_paths = []
new_bbs = []
train_path_resized = 'resized'
for index, row in anno.iterrows():
    new_path,new_bb = resize_image_bb(row['filename'], train_path_resized, create_bb_array(row.values),400)
    anno.at[index, "x1"] = new_bb[1]
    anno.at[index, "x2"] = new_bb[3]
    anno.at[index, "y1"] = new_bb[0]
    anno.at[index, "y2"] = new_bb[2]
    new_paths.append(new_path)
    new_bbs.append(new_bb)
anno['new_path'] = new_paths
anno['new_bb'] = new_bbs

resized/01/image.000935.jp2
resized/01/image.000936.jp2
resized/01/image.000937.jp2
resized/05/image.000938.jp2
resized/05/image.000939.jp2
resized/07/image.000940.jp2
resized/02/image.000941.jp2
resized/02/image.000942.jp2
resized/02/image.000943.jp2
resized/01/image.000944.jp2
resized/01/image.000945.jp2
resized/01/image.000946.jp2
resized/00/image.000947.jp2
resized/00/image.000948.jp2
resized/00/image.000949.jp2
resized/05/image.000950.jp2
resized/05/image.000951.jp2
resized/05/image.000952.jp2
resized/05/image.000953.jp2
resized/05/image.000954.jp2
resized/05/image.000955.jp2
resized/01/image.000956.jp2
resized/01/image.000957.jp2
resized/01/image.000958.jp2
resized/05/image.000959.jp2
resized/05/image.000960.jp2
resized/05/image.000961.jp2
resized/07/image.000962.jp2
resized/07/image.000963.jp2
resized/07/image.000964.jp2
resized/07/image.000965.jp2
resized/07/image.000966.jp2
resized/07/image.000967.jp2
resized/07/image.000968.jp2
resized/07/image.000969.jp2
resized/07/image.000

KeyboardInterrupt: 

In [ ]:
anno

In [ ]:
def create_corner_rect(bb, color='red'):
    bb = np.array(bb, dtype=np.float32)
    return plt.Rectangle((bb[1], bb[0]), bb[3]-bb[1], bb[2]-bb[0], color=color,
                         fill=False, lw=3)

def show_corner_bb(im, bb):
    plt.figure(figsize=(15,10))
    plt.imshow(im)
    plt.gca().add_patch(create_corner_rect(bb))

In [ ]:
im_num = random.randint(0, 900)
im = cv2.imread(str(anno.values[im_num][9]))
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
show_corner_bb(im, anno.values[im_num][10])